In [ ]:

phytp=pd.read_csv('/lustre/scratch126/cellgen/team298/ab72/accelerate/PHYTOPLANKTON.csv')

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


df_grouped = phytp.groupby(["siteid", "taxaname"]).size().reset_index(name="count")

# Create a stacked bar plot
plt.figure(figsize=(12, 6))
sns.barplot(x="siteid", y="count", hue="taxaname", data=df_grouped, palette="Set2")

#  Customize the plot
plt.xticks(rotation=45)  # Rotate x-axis labels for readability
plt.title("Taxaname Distribution Across Site IDs")
plt.xlabel("Site ID")
plt.ylabel("Count of Taxaname")
plt.legend(title="Taxaname", bbox_to_anchor=(1.05, 1), loc='upper left')  # Move legend outside
plt.show()


In [30]:
merged2=pd.read_csv('/lustre/scratch126/cellgen/team298/ab72/accelerate/WATER_METADATA_PHYTOPLANKTON.csv')

/tmp/ipykernel_1071276/3671086491.py:1: DtypeWarning: Columns (23,24,33,35,39,40,41,42,43,44,45,46,47,48,49,58) have mixed types. Specify dtype option on import or set low_memory=False.
  merged2=pd.read_csv('/lustre/scratch126/cellgen/team298/ab72/accelerate/WATER_METADATA_PHYTOPLANKTON.csv')


In [31]:
subset_pandas = merged2[merged2["siteid"] == "E-A10HS"]

#  Display the subset
print(subset_pandas.head())

    siteid                                    sitename country        seaname  \
0  E-A10HS  Ondarroa (Embarcadero). High Tide. Surface      ES  Bay of Biscay   
1  E-A10HS  Ondarroa (Embarcadero). High Tide. Surface      ES  Bay of Biscay   
2  E-A10HS  Ondarroa (Embarcadero). High Tide. Surface      ES  Bay of Biscay   
3  E-A10HS  Ondarroa (Embarcadero). High Tide. Surface      ES  Bay of Biscay   
4  E-A10HS  Ondarroa (Embarcadero). High Tide. Surface      ES  Bay of Biscay   

   decimallatitude  decimallongitude datecollected  Temperature    pH  \
0        43.319883         -2.420973    2002-04-24         16.5  8.34   
1        43.319883         -2.420973    2002-04-24         16.5  8.34   
2        43.319883         -2.420973    2002-04-24         16.5  8.34   
3        43.319883         -2.420973    2002-04-24         16.5  8.34   
4        43.319883         -2.420973    2002-04-24         16.5  8.34   

   Dissolved oxygen  ...        sampleid   basisofrecord  \
0             

In [ ]:
import pandas as pd
import statsmodels.api as sm

# Get unique species
unique_species = subset_pandas["taxaname"].unique()

#  Dictionary to store results
regression_results = {"Temperature": {}, "pH": {}}

#  Loop through each species and fit two regression models
for species in unique_species:
    df_species = subset_pandas[subset_pandas["taxaname"] == species][["parameter_value", "Temperature", "pH"]].dropna()
    
    # Ensure enough data points for regression
    if len(df_species) < 2:
        print(f"Skipping {species} (not enough data)")
        continue

    ##  1. Regression: `parameter_value` vs `Temperature`
    X_temp = df_species["Temperature"]
    y = df_species["parameter_value"]
    X_temp = sm.add_constant(X_temp)  # Add intercept

    model_temp = sm.OLS(y, X_temp).fit()
    regression_results["Temperature"][species] = model_temp.summary()
    
    print(f"🔹 Regression Results for {species} (Temperature)")
    print(model_temp.summary())
    print("\n" + "="*60 + "\n")

    ## 2. Regression: `parameter_value` vs `pH`
    X_pH = df_species["pH"]
    X_pH = sm.add_constant(X_pH)  # Add intercept

    model_pH = sm.OLS(y, X_pH).fit()
    regression_results["pH"][species] = model_pH.summary()

    print(f"🔹 Regression Results for {species} (pH)")
    print(model_pH.summary())
    print("\n" + "="*60 + "\n")


In [ ]:
results_list = []

for species in regression_results["Temperature"]:
    temp_coef = regression_results["Temperature"][species].tables[1].data[1][1]  # Extract temperature coefficient
    temp_p_value = regression_results["Temperature"][species].tables[1].data[1][4]  # Extract p-value
    
    pH_coef = regression_results["pH"][species].tables[1].data[1][1]  # Extract pH coefficient
    pH_p_value = regression_results["pH"][species].tables[1].data[1][4]  # Extract p-value

    results_list.append([species, temp_coef, temp_p_value, pH_coef, pH_p_value])

df_results = pd.DataFrame(results_list, columns=["Species", "Temp Coefficient", "Temp P-Value", "pH Coefficient", "pH P-Value"])
print(df_results)


In [ ]:
df_results.to_csv('/lustre/scratch126/cellgen/team298/ab72/accelerate/temp_ph_species.csv')

In [ ]:
#rank coefficientsfor each species

import pandas as pd
import statsmodels.api as sm

#  Get unique species
unique_species = subset_pandas["taxaname"].unique()

#  Dictionary to store coefficients and p-values
results_list = []

#  Loop through each species and fit two regression models
for species in unique_species:
    df_species = subset_pandas[subset_pandas["taxaname"] == species][["parameter_value", "Temperature", "pH"]].dropna()

    # Ensure enough data points for regression
    if len(df_species) < 2:
        continue  # Skip species with insufficient data

    ##  1. Regression: `parameter_value` vs `Temperature`
    X_temp = sm.add_constant(df_species["Temperature"])
    y = df_species["parameter_value"]

    model_temp = sm.OLS(y, X_temp).fit()
    temp_coef = model_temp.params["Temperature"]  # Extract Temperature coefficient
    temp_p_value = model_temp.pvalues["Temperature"]  # Extract Temperature p-value

    ##  2. Regression: `parameter_value` vs `pH`
    X_pH = sm.add_constant(df_species["pH"])
    model_pH = sm.OLS(y, X_pH).fit()
    pH_coef = model_pH.params["pH"]  # Extract pH coefficient
    pH_p_value = model_pH.pvalues["pH"]  # Extract pH p-value

    #  Store results
    results_list.append([species, temp_coef, temp_p_value, pH_coef, pH_p_value])

#  Convert to DataFrame
df_results = pd.DataFrame(results_list, columns=["Species", "Temp Coefficient", "Temp P-Value", "pH Coefficient", "pH P-Value"])

#  Sort by strongest effects (absolute coefficients)
df_sorted_temp = df_results.sort_values(by="Temp Coefficient", key=abs, ascending=False)
df_sorted_pH = df_results.sort_values(by="pH Coefficient", key=abs, ascending=False)

#  Display top species affected by Temperature and pH
print("🔹 Top 10 Species Most Affected by Temperature:")
print(df_sorted_temp.head(10))

print("\n🔹 Top 10 Species Most Affected by pH:")
print(df_sorted_pH.head(10))


In [ ]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

#  Get unique species
unique_species = subset_pandas["taxaname"].unique()

#  Set up figure size and layout
num_species = len(unique_species)
num_cols = 3  # Number of columns in the plot grid
num_rows = (num_species // num_cols) + (num_species % num_cols > 0)  # Calculate rows

fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, num_rows * 4))
axes = axes.flatten()  # Flatten in case of multiple rows

#  Loop through each species and plot regression
for i, species in enumerate(unique_species):
    df_species = subset_pandas[subset_pandas["taxaname"] == species][["parameter_value", "Temperature"]].dropna()

    # Ensure enough data points for regression
    if len(df_species) < 2:
        continue  # Skip species with insufficient data

    # Fit Linear Regression
    X = sm.add_constant(df_species["Temperature"])
    y = df_species["parameter_value"]
    model = sm.OLS(y, X).fit()

    # Plot scatter + regression line
    sns.regplot(x=df_species["Temperature"], y=df_species["parameter_value"], ax=axes[i], 
                scatter_kws={"alpha": 0.5}, line_kws={"color": "red"})
    
    # Titles and labels
    axes[i].set_title(f"{species}")
    axes[i].set_xlabel("Temperature")
    axes[i].set_ylabel("Abundance (parameter_value)")

#  Adjust layout
plt.tight_layout()
plt.show()
